In [3]:
from dataset import tweetProccess
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk, re
from sklearn.feature_extraction.text import CountVectorizer
import math

'''docs = ['this this this book',
        'this cat good',
        'cat good shit',
       'cat bad',
       'cat killer',
       'cat injustice']'''

treat = tweetProccess.FileTreat()

doc_num = 300

docs = treat.getBadTweets()[:doc_num]

#print(stopwords.words('english')[0:3])

def tokenize(text):
    text = word_tokenize(text)
    text = [i for i in text if i not in stopwords.words('english')]
    
    return text
    

def get_coocurrence_matrix():
    cv = CountVectorizer(tokenizer=tokenize, ngram_range=(1,1))
    com = cv.fit_transform(docs)
    
    #print(com.toarray())
    
    co_matrix = (com.T * com)
    return co_matrix.toarray()

def get_tf():
    '''gets the term frequency matrix
    returns the vocabulary and the tf matrix'''
    cv = CountVectorizer(tokenizer=tokenize)
    tf = cv.fit_transform(docs)
    tf = tf.T.toarray()
    
    tf = [sum(i) for i in tf]
    
    
    return cv.get_feature_names(), tf

def getdictwords(path):
    f = open(path, 'r', encoding="ISO-8859-1")
    lines = f.readlines()
    
    return [i.replace('\n', '') for i in lines]

positive_words = getdictwords('dataset/src/positive-words.txt')
negative_words = getdictwords('dataset/src/negative-words.txt')

def get_reference(names):
    '''gets a dictionary word->index'''
    d = {}
    for i in range(len(names)):
        d[names[i]] = i
        
    return d
    
def get_pmi(coocurrence_matrix, tf):
    pmi = []
    for idx in range(len(tf)):
        pmi.append([])
        for jdx in range(len(tf)):
            #gets the coocurrence probability
            num = coocurrence_matrix[idx][jdx] / doc_num
            #gets the product of the first term and second term probability
            den = (tf[idx] / doc_num) * (tf[jdx] / doc_num)
            #gets the measure of the dependence of these two words
            dep =  num/den
            pmi[idx].append(0 if dep == 0 else math.log(dep, 2))
    
    return pmi

def filter_structure(text):
    '''gets the patern of words described in the article www.aclweb.org/anthology/P02-1053.pdf'''
    tokens = word_tokenize(text)
    tags = nltk.pos_tag(tokens)
    words = []
    
    for i in range(len(tags) - 1):
        if tags[i][1] == 'JJ' and (tags[i + 1][1] == 'NN' or tags[i + 1][1] == 'NNS'):
            words.extend([tags[i][0], tags[i + 1][0]])
        elif tags[i][1] == 'JJ' and tags[i + 1][1] == 'JJ':
            words.extend([tags[i][0], tags[i + 1][0]])
        elif (tags[i][1] == 'RB' or tags[i][1] == 'RBR' or tags[i][1] == 'RBS') and (tags[i + 1][1] == 'JJ'):
            words.extend([tags[i][0], tags[i + 1][0]])
        elif (tags[i][1] == 'NN' or tags[i][1] == 'NNS') and tags[i + 1][1] == 'JJ':
            words.extend([tags[i][0], tags[i + 1][0]])
        elif (tags[i][1] == 'RB' or tags[i][1] == 'RBR' or tags[i][1] == 'RBS') and (tags[i + 1][1] == 'VB' or tags[i + 1][1] == 'VBD' or tags[i + 1][1] == 'VBN' or tags[i + 1][1] == 'VBG'):
            words.extend([tags[i][0], tags[i + 1][0]])
    
    return words

def filter_vocab(vocab, words):
    '''gets the intersection between vocab and words with the id in vocab'''
    intersect = []
    for word in words:
        if word in vocab:
            intersect.append(word)
    return intersect
            
def get_semantic_orientation(pmi, phrase, positive_id, negative_id, ref):
    #gets the words that matter
    words = filter_structure(phrase)
    negative = .0
    positive = .0
    #for each word sum the positive and negative association with positive and negative words
    for word in words:
        if word in ref:
            positive += sum(pmi[ref[word.lower()]][ref[i]]  for i in positive_id)
            negative += sum(pmi[ref[word.lower()]][ref[i]]  for i in negative_id)
        
    if len(words) > 0:
        positive /= len(words)/2
        negative /= len(words)/2
    return positive - negative
    
    

In [4]:
#gets the tf and vocab
vocab, tf = get_tf()
#print(vocab)
#gets coocurrence matrix
co_matrix = get_coocurrence_matrix()
#gets the PMI
pmi = get_pmi(co_matrix, tf)
print(len(co_matrix))
print(len(co_matrix[0]))
print(len(tf))
#gets the column reference
ref = get_reference(vocab)
#gets positive and negative words that are in our vocabulary
positive_id, negative_id = filter_vocab(vocab, positive_words), filter_vocab(vocab, negative_words)

#print(positive_id)
#print(negative_id)

    

1206
1206
1206


In [5]:
#idx = 187
print(docs[173])
for i in range(len(docs)):
    print(str(i))
    print(get_semantic_orientation(pmi, docs[i],positive_id, negative_id, ref))
    print('==========')

Do we need a better reason to #BellLetsTalk about it? Open up discussion of #mentalhealth. Our children are watchin… https://t.co/6bKKGJbs35
0
0.0
1
7.303780748177103
2
0.0
3
38.01474989510092
4
1.0859587255427101
5
0.0
6
0.0
7
-11.326120178074865
8
0.0
9
0.0
10
13.520916457955941
11
0.0
12
0.0
13
-28.91527476198352
14
9.813781191217037
15
-8.328980308208859
16
4.50136209252228
17
0.0
18
0.0
19
0.0
20
15.75141768612893
21
-16.610450894660875
22
0.0
23
-4.467628240974626
24
0.0
25
7.290795106582623
26
0.0
27
0.0
28
0.0
29
15.75141768612893
30
0.0
31
0.0
32
-20.101898780878518
33
10.880709064584432
34
-12.223644962971527
35
0.0
36
-9.62778782114707
37
0.0
38
0.0
39
0.0
40
28.91527476198352
41
0.0
42
26.008384166375006
43
0.0
44
0.0
45
14.15619517663255
46
0.0
47
0.0
48
0.0
49
48.905737020361265
50
12.638599478217111
51
-18.307713265629317
52
0.0
53
0.0
54
0.0
55
0.0
56
0.0
57
0.0
58
48.905737020361265
59
0.0
60
0.0
61
0.0
62
0.0
63
0.0
64
0.0
65
0.0
66
0.0
67
0.0
68
0.0
69
-11.5526003247